# SEI Task

GOAL: The script should create csv files containing the flows belonging to the different branches with additional columns (computed by the decision tree) branch and logistics_hub, either as a single file with all flows labeled with their branch and logistics hub, or as separate files for each branch.

 - Branch 1: if the tax id of the shipment (the "exporter CNPJ") corresponds to a farm (i.e. a CNPJ of level 1), assign the shipment to branch 1 and accept the tax municipality as the "logistics hub"
 - Branch 2: if the tax id of the shipment corresponds to a silo (i.e. a CNPJ of level 2), assign the shipment to branch 2 and accept the tax municipality as the "logistics hub"
 - Branch 3: if the tax id of the shipment does not correspond to a farm or silo, but the exporting company owns a farm in the tax municipality (i.e. there is a CNPJ of level 1 that has the same first 8 digits as the exporter CNPJ in the tax municipality of the exporter CNPJ), assign the shipment to branch 3 and accept the tax municipality as the "logistics hub"
 - Branch 4: if the tax id of the shipment does not correspond to a farm or silo, but the exporting company owns a silo in the tax municipality(i.e. there is a CNPJ of level 2 that has the same first 8 digits as the exporter CNPJ in the tax municipality of the exporter CNPJ), assign the shipment to branch 4 and accept the tax municipality as the "logistics hub"

In [1]:
import pandas as pd


## 1. Loading exports data - individual shipments exported from Brazil

In [2]:
data = pd.read_csv('data.csv', sep=';')

In [3]:
data.head()

,product_type,exporter_cnpj,vol,port_of_export,country_of_destination
0,SOYBEAN CAKE,12377000160,1.912534e+07,SANTOS,INDONESIA
1,SOYBEANS,80671000100,1.881031e+07,PARANAGUA,BANGLADESH
2,SOYBEANS,80671000100,2.080025e+07,PARANAGUA,CHINA (MAINLAND)
3,SOYBEAN OIL,80671000100,1.978489e+04,PARANAGUA,INDIA
4,SOYBEAN CAKE,80671000100,3.605974e+07,SANTOS,INDONESIA


- product_type, the type of soy product exported
- exporter_cnpj, the tax ID number of the shipment; this is a 14-digit number, the first 8 digits correspond to the company, and the remaining 6 digits identify the exact facility where the products were processed before being exported.
- vol, the amount of product exported in kilograms
- port_of_export, the name of the port where the products are loaded onto a ship and exported
- country_of_destination

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039 entries, 0 to 2038
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_type            2039 non-null   object 
 1   exporter_cnpj           2039 non-null   int64  
 2   vol                     2039 non-null   float64
 3   port_of_export          2039 non-null   object 
 4   country_of_destination  2039 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 79.8+ KB


## 2. Loading cnpj data - contains tax information on companies involved in the trade of soy in Brazil and their facilities

In [5]:
cnpj = pd.read_csv('cnpj.csv', sep=';')

-cnpj, the tax ID number of the facility owned by the company

-tax_municipality, a code corresponding to the city where the facility is located

-level, the type of activity the tax record corresponds to; "1" is for soy farming, "2" is for storage of soy in a silo.

In [6]:
cnpj.head()

,level,cnpj8,tax_municipality,cnpj
0,1,78956968,BR-4114807,78956968005495
1,1,78956968,BR-4111100,78956968000426
2,1,77294254,BR-1503606,77294254008098
3,1,77294254,BR-5107065,77294254004939
4,1,77294254,BR-5102686,77294254005153


In [16]:
cnpj['level'].unique()

array([1, 2, 3, 4, 5])

In [7]:
cnpj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   level             557 non-null    int64 
 1   cnpj8             557 non-null    int64 
 2   tax_municipality  557 non-null    object
 3   cnpj              557 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 17.5+ KB


## 3. Merging the data

In [ ]:
# Create an exporter cnpj8 
result['exporter_cnpj_8'] = exporter_cnpj

In [8]:
# Joining the tables to enrich with tax information
result = data.merge(cnpj, left_on='exporter_cnpj', right_on= 'cnpj')

In [9]:
result.head()

,product_type,exporter_cnpj,vol,port_of_export,country_of_destination,level,cnpj8,tax_municipality,cnpj
0,SOYBEAN CAKE,12377000160,1.912534e+07,SANTOS,INDONESIA,2,12377,BR-5218805,12377000160
1,SOYBEANS,80671000100,1.881031e+07,PARANAGUA,BANGLADESH,1,80671,BR-5211503,80671000100
2,SOYBEANS,80671000100,2.080025e+07,PARANAGUA,CHINA (MAINLAND),1,80671,BR-5211503,80671000100
3,SOYBEAN OIL,80671000100,1.978489e+04,PARANAGUA,INDIA,1,80671,BR-5211503,80671000100
4,SOYBEAN CAKE,80671000100,3.605974e+07,SANTOS,INDONESIA,1,80671,BR-5211503,80671000100


In [10]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1777 entries, 0 to 1776
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_type            1777 non-null   object 
 1   exporter_cnpj           1777 non-null   int64  
 2   vol                     1777 non-null   float64
 3   port_of_export          1777 non-null   object 
 4   country_of_destination  1777 non-null   object 
 5   level                   1777 non-null   int64  
 6   cnpj8                   1777 non-null   int64  
 7   tax_municipality        1777 non-null   object 
 8   cnpj                    1777 non-null   int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 138.8+ KB


In [11]:
# some data coudn't be found in cnpj table
result[result['tax_municipality'].isnull()]

,product_type,exporter_cnpj,vol,port_of_export,country_of_destination,level,cnpj8,tax_municipality,cnpj


### 4. Building the logic

In [25]:
# assign exporter_cnpj_8
result['exporter_cnpj_8'] = result['exporter_cnpj'].astype(str).str[:7]
result['exporter_cnpj_8'] = result['exporter_cnpj_8'].astype(int)

In [62]:
# generate cnpjs of those who own a farm
owns_a_farm_data = cnpj[cnpj['level'] == 1]
owns_a_farm = data['cnpj8'].tolist()

In [80]:
# generate cnpjs of those who own a farm
owns_a_silo_data = cnpj[cnpj['level'] == 2]
owns_a_silo = data['cnpj8'].tolist()

In [87]:
def assign_brunch(x):
    if(x['level'] == 1):
        return 'Branch 1'
    elif (x['level'] == 2):
        return 'Branch 2'
    # company owns a farm in the tax municipality
    elif (x['level'] > 2 and x['exporter_cnpj_8'] in owns_a_farm ):
        return 'Branch 3'
     # company owns a silo in the tax municipality
    elif (x['level'] > 2 and x['exporter_cnpj_8'] in owns_a_silo ):
        return 'Branch 4'
    else:
        return 'out of scope'

In [88]:
result['branch'] = result.apply(assign_brunch, axis=1)

In [90]:
result['branch'].unique()

array(['Branch 2', 'Branch 1', 'out of scope', 'Branch 3'], dtype=object)

In [92]:
# those with branches assigned have a tax municipality type as logistics hub 
# lets filter out those values and save them
result.loc[result['branch'] != 'out of scope', 'tax_municipality_type'] = 'logistics hub'

In [93]:
result.head()

,product_type,exporter_cnpj,vol,port_of_export,country_of_destination,level,cnpj8,tax_municipality,cnpj,exporter_cnpj_8,branch,tax_municipality_type
0,SOYBEAN CAKE,12377000160,1.912534e+07,SANTOS,INDONESIA,2,12377,BR-5218805,12377000160,1237700,Branch 2,logistics hub
1,SOYBEANS,80671000100,1.881031e+07,PARANAGUA,BANGLADESH,1,80671,BR-5211503,80671000100,8067100,Branch 1,logistics hub
2,SOYBEANS,80671000100,2.080025e+07,PARANAGUA,CHINA (MAINLAND),1,80671,BR-5211503,80671000100,8067100,Branch 1,logistics hub
3,SOYBEAN OIL,80671000100,1.978489e+04,PARANAGUA,INDIA,1,80671,BR-5211503,80671000100,8067100,Branch 1,logistics hub
4,SOYBEAN CAKE,80671000100,3.605974e+07,SANTOS,INDONESIA,1,80671,BR-5211503,80671000100,8067100,Branch 1,logistics hub


In [94]:
result.to_csv('result.csv', index=False)